In [27]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from pathlib import Path
# Step 1: Create a session and load the page
url1 = 'https://www.nba.com/stats/teams/shots-closest-defender?CloseDefDistRange=0-2+Feet+-+Very+Tight&PerMode=Totals'
url2 = 'https://www.nba.com/stats/teams/shots-closest-defender?CloseDefDistRange=2-4+Feet+-+Tight&PerMode=Totals'
url3 = 'https://www.nba.com/stats/teams/shots-closest-defender?CloseDefDistRange=4-6+Feet+-+Open&PerMode=Totals'
url4 = 'https://www.nba.com/stats/teams/shots-closest-defender?CloseDefDistRange=6%2B+Feet+-+Wide+Open&PerMode=Totals'
url_list = [url1,url2,url3,url4]
#url_list =[url +'&SeasonType=Playoffs' for url in url_list]
url_list =[url +'&SeasonType=Regular+Season'for url in url_list]

def get_tables(url_list):
    data = []
    xpath = '//*[@id="__next"]/div[2]/div[2]/div[3]/section[2]/div/div[2]/div[3]/table'
    driver = webdriver.Chrome()
    for url in url_list:
        
        driver.get(url)
        print(url)
        # Wait for the page to fully load
        driver.implicitly_wait(20)
        element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, xpath)))

        # Step 2: Parse HTML code and grab tables with Beautiful Soup
        soup = BeautifulSoup(driver.page_source, 'lxml')

        tables = soup.find_all('table')

        # Step 3: Read tables with Pandas read_html()
        dfs = pd.read_html(str(tables))

        #print(f'Total tables: {len(dfs)}')
        #print(dfs[2].head())

        
        df= dfs[-1]
        #print(df)
        drop = ['Unnamed: 16_level_1', 'Unnamed: 17_level_1', 'Unnamed: 18_level_1']
        df.columns = df.columns.droplevel()
        df = df.drop(columns = drop)
        data.append(df)
    driver.close()
    return data
#url_list = [url1]#
def multiyear_shooting(url_list,team_round=0,playoffs = True):
    df_list = []

    for i in range(2013,2023):
        year = i+1
        season = '&Season='+str(i)+'-'+str(i+1 - 2000)
        year_url = [url+season for url in url_list]
        if team_round!=0:
            year_url = [url + '&PORound='+str(team_round) for url in year_url]
        frames = get_tables(year_url)

 
        path = str(i+1)+'/playoff_shooting/round'+str(team_round)+'/'
        if playoffs == False:
            path = str(i+1)+'/team_shooting/'
        output_dir = Path(path)
        output_dir.mkdir(parents=True, exist_ok=True)
        #terms = ['data/teampullup.csv','data/teamcatchshoot.csv','data/teamundersix.csv','data/teamiso.csv','data/teamtransition.csv']
        terms = ['very_tight','tight','open','wide_open']
        
        
        for k in range(len(terms)):
            
            frames[k]['shot_coverage'] = terms[k]
            if playoffs == True:
                frames[k]['round'] = team_round
            frames[k]['year'] = year
        df = pd.concat(frames)
        df.to_csv(path+'team_shooting.csv',index = False)
        df_list.append(df)
    final_df = pd.concat(df_list)
    return final_df

df = multiyear_shooting(url_list,playoffs=False)
df

https://www.nba.com/stats/teams/shots-closest-defender?CloseDefDistRange=0-2+Feet+-+Very+Tight&PerMode=Totals&SeasonType=Regular+Season&Season=2013-14
https://www.nba.com/stats/teams/shots-closest-defender?CloseDefDistRange=2-4+Feet+-+Tight&PerMode=Totals&SeasonType=Regular+Season&Season=2013-14
https://www.nba.com/stats/teams/shots-closest-defender?CloseDefDistRange=4-6+Feet+-+Open&PerMode=Totals&SeasonType=Regular+Season&Season=2013-14
https://www.nba.com/stats/teams/shots-closest-defender?CloseDefDistRange=6%2B+Feet+-+Wide+Open&PerMode=Totals&SeasonType=Regular+Season&Season=2013-14
https://www.nba.com/stats/teams/shots-closest-defender?CloseDefDistRange=0-2+Feet+-+Very+Tight&PerMode=Totals&SeasonType=Regular+Season&Season=2014-15
https://www.nba.com/stats/teams/shots-closest-defender?CloseDefDistRange=2-4+Feet+-+Tight&PerMode=Totals&SeasonType=Regular+Season&Season=2014-15
https://www.nba.com/stats/teams/shots-closest-defender?CloseDefDistRange=4-6+Feet+-+Open&PerMode=Totals&Season

,TEAM,GP,G,Freq%,FGM,FGA,FG%,eFG%,2FG Freq%,2FGM,2FGA,2FG%,3FG Freq%,3PM,3PA,3P%,shot_coverage,year
0,Detroit Pistons,82,82,23.1,843,1554,54.2,54.4,22.7,839,1529,54.9,0.4,4,25,16.0,very_tight,2014
1,Philadelphia 76ers,82,82,22.5,762,1479,51.5,51.7,22.2,756,1461,51.7,0.3,6,18,33.3,very_tight,2014
2,Minnesota Timberwolves,82,82,20.0,707,1342,52.7,52.8,19.8,705,1328,53.1,0.2,2,14,14.3,very_tight,2014
3,Indiana Pacers,82,82,21.0,673,1302,51.7,51.8,20.7,670,1283,52.2,0.3,3,19,15.8,very_tight,2014
4,Memphis Grizzlies,82,82,20.2,689,1276,54.0,54.0,20.0,688,1263,54.5,0.2,1,13,7.7,very_tight,2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25,Washington Wizards,82,82,19.9,594,1415,42.0,58.0,3.3,142,235,60.4,16.6,452,1180,38.3,wide_open,2023
26,Denver Nuggets,82,82,19.8,628,1406,44.7,61.2,3.7,164,264,62.1,16.1,464,1142,40.6,wide_open,2023
27,LA Clippers,82,82,19.7,615,1392,44.2,62.3,2.6,111,181,61.3,17.2,504,1211,41.6,wide_open,2023
28,Charlotte Hornets,82,82,18.8,567,1390,40.8,55.5,3.4,159,249,63.9,15.5,408,1141,35.8,wide_open,2023


In [28]:
df.to_csv('team_shooting_rs.csv',index = False)

In [9]:
master = master.sort_values(by=['year','round','shot_coverage','TEAM'])
master

,TEAM,GP,G,Freq%,FGM,FGA,FG%,eFG%,2FG Freq%,2FGM,2FGA,2FG%,3FG Freq%,3PM,3PA,3P%,shot_coverage,round,year
2,Atlanta Hawks,7,7,31.6,62,160,38.8,47.5,14.6,34,74,45.9,17.0,28,86,32.6,open,1,2014
5,Brooklyn Nets,7,7,28.2,63,142,44.4,51.1,15.5,44,78,56.4,12.7,19,64,29.7,open,1,2014
15,Charlotte Bobcats,4,4,25.1,30,71,42.3,51.4,12.4,17,35,48.6,12.7,13,36,36.1,open,1,2014
13,Chicago Bulls,5,5,27.8,40,100,40.0,47.0,15.6,26,56,46.4,12.2,14,44,31.8,open,1,2014
6,Dallas Mavericks,7,7,24.2,58,141,41.1,46.8,14.8,42,86,48.8,9.5,16,55,29.1,open,1,2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,Miami Heat,5,5,44.8,73,197,37.1,38.8,35.9,66,158,41.8,8.9,7,39,17.9,tight,4,2023
0,Denver Nuggets,5,5,10.1,20,40,50.0,50.0,9.8,20,39,51.3,0.3,0,1,0.0,very_tight,4,2023
1,Miami Heat,5,5,5.5,8,24,33.3,33.3,5.5,8,24,33.3,0.0,0,0,-,very_tight,4,2023
1,Denver Nuggets,5,5,13.9,28,55,50.9,68.2,3.0,9,12,75.0,10.8,19,43,44.2,wide_open,4,2023


In [10]:
master.to_csv('playoff_shooting.csv',index = False)

In [22]:
master.sort_values(by=['shot_coverage','3P%'],ascending = False).head(40)

,TEAM,GP,G,Freq%,FGM,FGA,FG%,eFG%,2FG Freq%,2FGM,2FGA,2FG%,3FG Freq%,3PM,3PA,3P%,shot_coverage,round,year
1,Miami Heat,7,7,15.2,52,90,57.8,79.4,3.9,13,23,56.5,11.3,39,67,58.2,wide_open,3,2023
7,Cleveland Cavaliers,4,4,22.2,37,67,55.2,76.1,5.3,9,16,56.3,16.9,28,51,54.9,wide_open,2,2017
15,Brooklyn Nets,4,4,17.0,30,53,56.6,77.4,3.8,8,12,66.7,13.1,22,41,53.7,wide_open,1,2022
11,Philadelphia 76ers,4,4,23.1,41,79,51.9,74.7,2.9,5,10,50.0,20.2,36,69,52.2,wide_open,1,2023
4,Portland Trail Blazers,5,5,21.9,53,93,57.0,76.9,5.2,16,22,72.7,16.7,37,71,52.1,wide_open,2,2016
13,Minnesota Timberwolves,5,5,19.9,36,76,47.4,66.4,5.2,7,20,35.0,14.7,29,56,51.8,wide_open,1,2018
3,Cleveland Cavaliers,4,4,29.8,49,95,51.6,74.7,2.8,5,9,55.6,27.0,44,86,51.2,wide_open,2,2016
11,LA Clippers,6,6,15.0,43,81,53.1,75.3,1.8,7,10,70.0,13.1,36,71,50.7,wide_open,1,2020
7,Brooklyn Nets,5,5,13.5,28,47,59.6,76.6,4.3,12,15,80.0,9.2,16,32,50.0,wide_open,2,2014
15,Cleveland Cavaliers,4,4,21.2,37,67,55.2,75.4,4.1,10,13,76.9,17.1,27,54,50.0,wide_open,1,2017


In [16]:
temp = master[master.shot_coverage =='open']
temp.sort_values(by=['shot_coverage','3P%'],ascending = False).head(30)

,TEAM,GP,G,Freq%,FGM,FGA,FG%,eFG%,2FG Freq%,2FGM,2FGA,2FG%,3FG Freq%,3PM,3PA,3P%,shot_coverage,round,year
6,Cleveland Cavaliers,4,4,29.2,49,93,52.7,66.1,15.0,24,48,50.0,14.1,25,45,55.6,open,2,2016
7,Miami Heat,7,7,21.5,60,117,51.3,64.1,11.4,30,62,48.4,10.1,30,55,54.5,open,1,2016
1,San Antonio Spurs,6,6,25.4,64,118,54.2,63.1,16.6,43,77,55.8,8.8,21,41,51.2,open,3,2014
1,San Antonio Spurs,5,5,23.4,40,82,48.8,61.6,11.7,19,41,46.3,11.7,21,41,51.2,open,4,2014
0,Cleveland Cavaliers,5,5,33.2,74,126,58.7,71.8,15.5,41,59,69.5,17.6,33,67,49.3,open,3,2017
7,Phoenix Suns,4,4,27.8,48,96,50.0,60.4,15.9,28,55,50.9,11.9,20,41,48.8,open,2,2021
13,San Antonio Spurs,4,4,27.7,40,80,50.0,58.8,17.6,26,51,51.0,10.0,14,29,48.3,open,1,2016
5,Utah Jazz,7,7,29.0,83,166,50.0,64.5,11.3,35,65,53.8,17.6,48,101,47.5,open,1,2020
12,Phoenix Suns,5,5,26.1,59,113,52.2,60.2,17.3,41,75,54.7,8.8,18,38,47.4,open,1,2023
6,San Antonio Spurs,6,6,26.1,53,117,45.3,55.6,14.7,29,66,43.9,11.4,24,51,47.1,open,1,2017


In [24]:
master[master.TEAM=='Miami Heat'][::-1].head(40).sort_values(by=['year','shot_coverage'],ascending = False)

,TEAM,GP,G,Freq%,FGM,FGA,FG%,eFG%,2FG Freq%,2FGM,2FGA,2FG%,3FG Freq%,3PM,3PA,3P%,shot_coverage,round,year
0,Miami Heat,5,5,15.5,30,68,44.1,58.1,4.1,11,18,61.1,11.4,19,50,38.0,wide_open,4,2023
1,Miami Heat,7,7,15.2,52,90,57.8,79.4,3.9,13,23,56.5,11.3,39,67,58.2,wide_open,3,2023
5,Miami Heat,6,6,18.8,39,97,40.2,56.7,2.3,7,12,58.3,16.5,32,85,37.6,wide_open,2,2023
14,Miami Heat,5,5,16.3,38,73,52.1,67.1,4.7,16,21,76.2,11.6,22,52,42.3,wide_open,1,2023
1,Miami Heat,5,5,5.5,8,24,33.3,33.3,5.5,8,24,33.3,0.0,0,0,-,very_tight,4,2023
0,Miami Heat,7,6,7.1,17,42,40.5,40.5,6.7,17,40,42.5,0.3,0,2,0.0,very_tight,3,2023
6,Miami Heat,6,6,5.8,16,30,53.3,55.0,5.4,15,28,53.6,0.4,1,2,50.0,very_tight,2,2023
13,Miami Heat,5,5,5.6,13,25,52.0,54.0,4.7,12,21,57.1,0.9,1,4,25.0,very_tight,1,2023
0,Miami Heat,5,5,44.8,73,197,37.1,38.8,35.9,66,158,41.8,8.9,7,39,17.9,tight,4,2023
0,Miami Heat,7,7,47.0,119,279,42.7,44.4,40.1,109,238,45.8,6.9,10,41,24.4,tight,3,2023
